In [91]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import time
from unidecode import unidecode
import numpy as np
from itertools import chain
final_qb_data = pd.read_csv("/Users/kmaran3/Dropbox/Darkhorse/final position group data/final_qb_data.csv")
final_qb_data['YearsBack'] = 2024 - final_qb_data['season'].astype(int)
final_qb_data.to_csv('/Users/kmaran3/Dropbox/Darkhorse/final position group data/final_qb_data.csv')
final_rb_data = pd.read_csv("/Users/kmaran3/Dropbox/Darkhorse/final position group data/final_rb_data.csv")
final_rb_data['YearsBack'] = 2024 - final_rb_data['season'].astype(int)
final_rb_data.to_csv('/Users/kmaran3/Dropbox/Darkhorse/final position group data/final_rb_data.csv')
final_wrte_data = pd.read_csv("/Users/kmaran3/Dropbox/Darkhorse/final position group data/final_wrte_data.csv")
final_wrte_data['YearsBack'] = 2024 - final_wrte_data['season'].astype(int)
final_wrte_data.to_csv('/Users/kmaran3/Dropbox/Darkhorse/final position group data/final_wrte_data.csv')

def dfMaker():
     #if ppr is 0 it is non ppr, if it is 1 it is half ppr, if its 2 it is full ppr. loops through
    for ppr in range(3):
        
        #read csv files into pandas
        oldQBStats = pd.read_csv("/Users/kmaran3/Dropbox/Darkhorse/final position group data/final_qb_data.csv")
        oldRBStats = pd.read_csv('/Users/kmaran3/Dropbox/Darkhorse/final position group data/final_rb_data.csv')
        oldWRTEStats = pd.read_csv('/Users/kmaran3/Dropbox/Darkhorse/final position group data/final_wrte_data.csv')
        currTeamsRoster = pd.read_csv("/Users/kmaran3/Dropbox/Darkhorse/currYearRoster.csv")
        pastTeamsRoster = pd.read_csv("/Users/kmaran3/Dropbox/Darkhorse/teamsPastRoster.csv")
        currAVs = pd.read_csv("/Users/kmaran3/Dropbox/Darkhorse/approximate value data/AVbyPositionGroup.csv")

        #make empty dfs and list to put players into
        completeDFQB = pd.DataFrame(columns= ["team", "position", "penalty", "GP", "player_display_name", "age", 'completions', 'attempts', 'passing_yards',
       'passing_tds', 'interceptions', 'sacks', 'sack_yards', 'sack_fumbles',
       'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch',
       'passing_first_downs', 'passing_epa', 'passing_2pt_conversions', 'pacr',
       'dakota', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles',
       'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa',
       'rushing_2pt_conversions', 'games', 'ppr_sh',
       'status', 'comp %', 'td:int', 'yards/attempts', 'yards/comp',
       'yards/carry', 'passer rating', "PPG", "oline", "rb", "wr", "qb", "te"])
        completeDFRB= pd.DataFrame(columns= ["team", "position", "penalty", "GP", "player_display_name", "age", 'carries', 'rushing_yards', 'rushing_tds',
       'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs',
       'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets',
       'receiving_yards', 'receiving_tds', 'receiving_fumbles',
       'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr_x', 'special_teams_tds', 'fantasy_points', 'games', 'tgt_sh',
       'ay_sh', 'yac_sh', 'wopr_y', 'ry_sh', 'rtd_sh', 'rfd_sh', 'rtdfd_sh',
       'dom', 'w8dom', 'yptmpa', 'ppr_sh', 'age', 'status', 'y/c', 'y/g',
       'c/g', 'y/rec', 'rec/g', 'y/tgt', 'catch %', 'touches', 'y/touch',
       'rrtd', "PPG", "oline", "rb", "wr", "qb", "te"])
        completeDFWRTE= pd.DataFrame(columns= ["team", "position", "penalty", "GP", "player_display_name", "age", 'carries', 'rushing_yards', 'rushing_tds',
       'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs',
       'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets',
       'receiving_yards', 'receiving_tds', 'receiving_fumbles',
       'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr_x', 'special_teams_tds', 'fantasy_points', 'games', 'tgt_sh',
       'ay_sh', 'yac_sh', 'wopr_y', 'ry_sh', 'rtd_sh', 'rfd_sh', 'rtdfd_sh',
       'dom', 'w8dom', 'yptmpa', 'ppr_sh', 'age', 'status', 'y/c', 'y/g',
       'c/g', 'y/rec', 'rec/g', 'y/tgt', 'catch %', 'touches', 'y/touch',
       'rrtd', "PPG", "oline", "rb", "wr", "qb", "te"])
        rookieList = []

        f = 0

        for index in range(len(currTeamsRoster)):

            f+=1
            #boolean to break if rookie
            breakBool = False

            #years back variable
            x = 1

            #penalty variable for injuries
            penalty = 0

            #current players place to store information
            individualDFQB = pd.DataFrame(columns= ["team", "position", "penalty", "GP", "player_display_name", "age", 'completions', 'attempts', 'passing_yards',
       'passing_tds', 'interceptions', 'sacks', 'sack_yards', 'sack_fumbles',
       'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch',
       'passing_first_downs', 'passing_epa', 'passing_2pt_conversions', 'pacr',
       'dakota', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles',
       'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa',
       'rushing_2pt_conversions', 'games', 'ppr_sh',
       'status', 'comp %', 'td:int', 'yards/attempts', 'yards/comp',
       'yards/carry', 'passer rating', "PPG", "oline", "rb", "wr", "qb", "te"])
            individualDFRB = pd.DataFrame(columns= ["team", "position", "penalty", "GP", "player_display_name", "age", 'carries', 'rushing_yards', 'rushing_tds',
       'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs',
       'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets',
       'receiving_yards', 'receiving_tds', 'receiving_fumbles',
       'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr_x', 'special_teams_tds', 'fantasy_points', 'games', 'tgt_sh',
       'ay_sh', 'yac_sh', 'wopr_y', 'ry_sh', 'rtd_sh', 'rfd_sh', 'rtdfd_sh',
       'dom', 'w8dom', 'yptmpa', 'ppr_sh', 'age', 'status', 'y/c', 'y/g',
       'c/g', 'y/rec', 'rec/g', 'y/tgt', 'catch %', 'touches', 'y/touch',
       'rrtd', "PPG", "oline", "rb", "wr", "qb", "te"])
            individualDFWRTE = pd.DataFrame(columns= ["team", "position", "penalty", "GP", "player_display_name", "age", 'carries', 'rushing_yards', 'rushing_tds',
       'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs',
       'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets',
       'receiving_yards', 'receiving_tds', 'receiving_fumbles',
       'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr_x', 'special_teams_tds', 'fantasy_points', 'games', 'tgt_sh',
       'ay_sh', 'yac_sh', 'wopr_y', 'ry_sh', 'rtd_sh', 'rfd_sh', 'rtdfd_sh',
       'dom', 'w8dom', 'yptmpa', 'ppr_sh', 'age', 'status', 'y/c', 'y/g',
       'c/g', 'y/rec', 'rec/g', 'y/tgt', 'catch %', 'touches', 'y/touch',
       'rrtd', "PPG", "oline", "rb", "wr", "qb", "te"])
            individualDFQB.loc[0] = [0] * len(individualDFQB.columns)
            individualDFRB.loc[0] = [0] * len(individualDFRB.columns)
            individualDFWRTE.loc[0] = [0] * len(individualDFWRTE.columns)

            #current players informaton
            year = currTeamsRoster.loc[index, 'Yrs']
            bday = currTeamsRoster.loc[index, 'BirthDate']
            pos = currTeamsRoster.loc[index, 'Pos']
            age = currTeamsRoster.loc[index, 'Age']
            name = currTeamsRoster.loc[index, 'Player']
            team = currTeamsRoster.loc[index, 'Team']

            games = 0

            #see if rookie
            if year == "Rook":
                #add player to rookie column
                dict = {}
                dict = {"Name": name, "Year": year, "Bday": bday, "Age": age, "Team": team}
                rookieList.append(dict)
                breakBool = True

                continue

            while True:
            
                #get all player rows from past year
                playerRowRoster = pastTeamsRoster.loc[(pastTeamsRoster['Player'] == name) & (pastTeamsRoster["YearsBack"] == x) & (pastTeamsRoster["BirthDate"] == bday)].copy()
                playerRowRoster = playerRowRoster.reset_index()
                
                #if there is no data for player it goes back to find data until years back (x) > 3
                if playerRowRoster.empty:
                    
                    if x>3:
                        if games>6:
                            #breaks if needed number of games is aqcuired.
                            break
                        elif (int(year)<4):
                            #add player to rookie column
                            dict = {}
                            dict = {"Name": name, "Year": year, "Bday": bday, "Age": age, "Team": team}
                            rookieList.append(dict)
                            breakBool = True

                            #breaks if cannot go back another year
                            break
                        else:
                            breakBool = True
                            break

                    x=x+1
                    continue
            
                for ind in range(len(playerRowRoster)):
                    teamCurr = playerRowRoster.loc[ind, "Team"]
                    numberCurr = playerRowRoster.loc[ind, "No."]
                    

                    if pos == "QB":
                        #gets correct player using team name, player name, and player number
                        playerRowStats = oldQBStats.loc[(oldQBStats["team"]== teamCurr) & (oldQBStats["player_display_name"]==name) & (oldQBStats["YearsBack"]==x)]
                        playerRowStats = playerRowStats.reset_index() 
                        playerRowStats = playerRowStats.fillna(0)

                        if playerRowStats.empty:
                            continue
                                                    
                        columns_to_update = [
                        'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions',
                        'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
                        'passing_yards_after_catch', 'passing_first_downs', 'passing_epa', 'passing_2pt_conversions',
                        'pacr', 'dakota', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles',
                        'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions', 'ppr_sh', 'comp %', 'td:int',
                        'yards/attempts', 'yards/comp', 'yards/carry', 'passer rating'
                        ]

                        for column in columns_to_update:
                            individualDFQB[column] = playerRowStats.loc[0, column] + individualDFQB[column]
    
                        
                        games = games + playerRowStats.loc[0, "GP"]
                    
                    elif pos == "RB":
                        playerRowStats = oldRBStats.loc[(oldRBStats["team"]== teamCurr) & (oldRBStats["player_display_name"]==name) & (oldRBStats["YearsBack"]==x)]
                        playerRowStats = playerRowStats.reset_index() 
                        playerRowStats = playerRowStats.fillna(0)

                        if playerRowStats.empty:
                            continue

                        columns_to_update = ['carries', 'rushing_yards', 'rushing_tds',
                        'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs',
                        'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets',
                        'receiving_yards', 'receiving_tds', 'receiving_fumbles',
                        'receiving_fumbles_lost', 'receiving_air_yards',
                        'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
                        'receiving_2pt_conversions','special_teams_tds',
                        'rrtd']

                        for column in columns_to_update:
                            individualDFRB[column] = playerRowStats.loc[0, column] + individualDFRB[column]
                            
                    elif pos in ['WR','TE']:
                        playerRowStats = oldWRTEStats.loc[(oldQBStats["team"]== teamCurr) & (oldWRTEStats["player_display_name"]==name) & (oldWRTEStats["YearsBack"]==x)]
                        playerRowStats = playerRowStats.reset_index() 
                        playerRowStats = playerRowStats.fillna(0)

                        if playerRowStats.empty:
                            continue

                        columns_to_update = ['carries', 'rushing_yards', 'rushing_tds',
                        'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs',
                        'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets',
                        'receiving_yards', 'receiving_tds', 'receiving_fumbles',
                        'receiving_fumbles_lost', 'receiving_air_yards',
                        'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
                        'receiving_2pt_conversions','special_teams_tds',
                        'rrtd']

                        for column in columns_to_update:
                            individualDFWRTE[column] = playerRowStats.loc[0, column] + individualDFWRTE[column]

                    if (x==1):
                        penalty = penalty + playerRowStats.loc[0, "GP"]   
                    elif (x==2):
                        penalty = penalty + (playerRowStats.loc[0, "GP"]*0.9)   
                    elif (x==3):
                        penalty = penalty + (playerRowStats.loc[0, "GP"]*0.8)   
                    elif (x==4):
                        penalty = penalty + (playerRowStats.loc[0, "GP"]*0.7)   
                
                if (games<6) and (x>3) and (int(year)<4):
                    #add player to rookie column
                    dict = {}
                    dict = {"Name": name, "Year": year, "Bday": bday, "Age": age, "Team": team}
                    rookieList.append(dict)
                    breakBool = True
                    break
                elif (games<6) and (x>3):
                    breakBool = True
                    break
                if (games>6) or (x>3):
                    break

                x+=1

                        #break if rookie
            if (breakBool == True):
                continue

            if pos == "QB":
                #make it all per game not total
                columns_to_update = [
                        'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions',
                        'sacks', 'sack_yards', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
                        'passing_yards_after_catch', 'passing_first_downs', 'passing_2pt_conversions',
                        'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles',
                        'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_2pt_conversions'
                ]
                for column in columns_to_update:
                        individualDFQB[column] = individualDFQB[column] / games
                individualDFQB["player_display_name"] = name
                individualDFQB["position"] = pos
                individualDFQB["team"] = team
                individualDFQB["GP"] = games
                individualDFQB["penalty"] = penalty/games

                individualDFQB.loc[:, "PPG"] = (individualDFQB["rushing_yards"]*0.1) + (individualDFQB["passing_yards"]*.04) + (individualDFQB["rushing_tds"]*6) + (individualDFQB["passing_tds"]*4) + (individualDFQB["rushing_fumbles"]*-2) + (individualDFQB["sack_fumbles"]*-2) + (individualDFQB["interceptions"]*-2)
                individualDFQB.loc[:, "PPG"] = individualDFQB["PPG"]
                
                #add other columns needed
                individualDFQB["age"] = age

                #locate row with correct team and pos and add it
                rowOL = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFQB['oline'] = rowOL['oline']
                
                rowRB = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFQB['rb'] = rowRB['rb']
                        
                rowWR = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFQB['wrte'] = rowWR['wrte']

                rowQB = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFQB['qb'] = rowQB['qb']

                rowTE = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFQB['dst'] = rowTE['dst']

                completeDFQB = pd.concat([completeDFQB, individualDFQB], ignore_index=True, join="inner")  
            elif pos == "RB":
                columns_to_update = ['carries', 'rushing_yards', 'rushing_tds',
                        'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs',
                        'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets',
                        'receiving_yards', 'receiving_tds', 'receiving_fumbles',
                        'receiving_fumbles_lost', 'receiving_air_yards',
                        'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
                        'receiving_2pt_conversions','special_teams_tds',
                        'rrtd']
                for column in columns_to_update:
                        individualDFRB[column] = individualDFRB[column] / games
                individualDFRB["player_display_name"] = name
                individualDFRB["position"] = pos
                individualDFRB["team"] = team
                individualDFRB["GP"] = games
                individualDFRB["penalty"] = penalty/games

                individualDFRB.loc[:, "PPG"] = (individualDFRB["rushing_yards"]*0.1) + (individualDFRB["rushing_tds"]*6) + (individualDFRB["rushing_fumbles"]*-2)
                individualDFRB.loc[:, "PPG"] = individualDFRB["PPG"]
                
                #add other columns needed
                individualDFRB["age"] = age

                #locate row with correct team and pos and add it
                rowOL = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFRB['oline'] = rowOL['oline']
                
                rowRB = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFRB['rb'] = rowRB['rb']
                        
                rowWR = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFRB['wrte'] = rowWR['wrte']

                rowQB = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFRB['qb'] = rowQB['qb']

                rowTE = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFRB['dst'] = rowTE['dst']

                completeDFRB = pd.concat([completeDFRB, individualDFRB], ignore_index=True, join="inner")  
            elif pos == "WRTE":
                columns_to_update = ['carries', 'rushing_yards', 'rushing_tds',
                        'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs',
                        'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets',
                        'receiving_yards', 'receiving_tds', 'receiving_fumbles',
                        'receiving_fumbles_lost', 'receiving_air_yards',
                        'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
                        'receiving_2pt_conversions','special_teams_tds',
                        'rrtd']
                for column in columns_to_update:
                        individualDFWRTE[column] = individualDFWRTE[column] / games
                individualDFWRTE["player_display_name"] = name
                individualDFWRTE["position"] = pos
                individualDFWRTE["team"] = team
                individualDFWRTE["GP"] = games
                individualDFWRTE["penalty"] = penalty/games

                individualDFWRTE.loc[:, "PPG"] = (individualDFWRTE["rushing_yards"]*0.1) + (individualDFQB["rushing_tds"]*6)
                individualDFWRTE.loc[:, "PPG"] = individualDFWRTE["PPG"]
                
                #add other columns needed
                individualDFWRTE["age"] = age

                #locate row with correct team and pos and add it
                rowOL = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFWRTE['oline'] = rowOL['oline']
                
                rowRB = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFWRTE['rb'] = rowRB['rb']
                        
                rowWRTE = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFWRTE['wrte'] = rowWR['wrte']

                rowQB = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFWRTE['qb'] = rowQB['qb']

                rowDST = currAVs.loc[currAVs['team'] == team].iloc[0]
                individualDFWRTE['dst'] = rowDST['dst']

                completeDFWRTE = pd.concat([completeDFWRTE, individualDFWRTE], ignore_index=True, join="inner")  
                #write into csv based on if ppr or not
        if ppr == 0:
            completeDFQB = completeDFQB.sort_values(by=['PPG'])
            completeDFQB.to_csv("QBDFForModelNonPPR.csv", encoding='utf-8', index=False)
            completeDFRB = completeDFRB.sort_values(by=['PPG'])
            completeDFRB.to_csv("RBDFForModelNonPPR.csv", encoding='utf-8', index=False)
            completeDFWRTE = completeDFWRTE.sort_values(by=['PPG'])
            completeDFWRTE.to_csv("WRTEDFForModelNonPPR.csv", encoding='utf-8', index=False)
        elif ppr == 1:
            completeDFQB = completeDFQB.sort_values(by=['PPG'])
            completeDFQB.to_csv("QBDFForModelHalfPPR.csv", encoding='utf-8', index=False)
        elif ppr == 2:
            completeDFQB = completeDFQB.sort_values(by=['PPG'])
            completeDFQB.to_csv("QBDFForModelPPR.csv", encoding='utf-8', index=False)

dfMaker()


In [81]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import time
from unidecode import unidecode
import numpy as np
from itertools import chain
import joblib
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore", message="X has feature names, but MLPRegressor was fitted without feature names")

#gets ppg back into normal form
def getScaleBack(df):
  #index of column
  column_index = df.columns.get_loc("PPG")

  #min value of column:
  min_value = df["PPG"].min()

  #scaling valye of column
  #scaling_factor = scaler.scale_[column_index]
  max_value = df["PPG"].max()

  #array to be used later to scale each data
  arr = [min_value, max_value]

  return arr

def scorer():

    #scaler to scale data
    scaler = MinMaxScaler()

    #if 0 its non ppr, if 1 its half ppr, if 2 its full ppr. loops through.

    for ppr in range(3):
        #dict of scores
        dictScores = {}

        if ppr == 0:
            #read in player stats df and make positional dfs
            qbs = pd.read_csv("/Users/kmaran3/Dropbox/Darkhorse/Models/QBDFForModelNonPPR.csv")

            qbModel = joblib.load("/Users/kmaran3/Dropbox/Darkhorse/Models/qbModelNonPPR.joblib")
        elif ppr == 1:
            #read in player stats df and make positional dfs
            qbs = pd.read_csv('/Users/kmaran3/Dropbox/Darkhorse/Models/QBDFForModelHalfPPR.csv')

            qbModel = joblib.load("/Users/kmaran3/Dropbox/Darkhorse/Models/qbModelHalfPPR.joblib")
        elif ppr == 2:        
            #read in player stats df and make positional dfs
            qbs = pd.read_csv("/Users/kmaran3/Dropbox/Darkhorse/Models/QBDFForModelPPR.csv")

            #loads models
            qbModel = joblib.load("/Users/kmaran3/Dropbox/Darkhorse/Models/qbModelPPR.joblib")

        #dictionary to easily call models
        modelsDict = {"QB": qbModel}

        #gets fantasypoints scale per each position
        scaleQB = getScaleBack(qbs)
        
        qbsScaled = qbs.copy()
        qbsScaled = qbsScaled.drop(columns=["GP", "position", "penalty", "player_display_name", "team"])        
        qbsScaled[qbsScaled.columns] = scaler.fit_transform(qbsScaled[qbsScaled.columns])

        #have these so it can iterate through easily
        allPosDFs = [qbs]
        allPosDfsScaled = [qbsScaled]
        scaleBack = [scaleQB]

        currPosDict = {}
        indPosArr = []

        #iterates through each position
        for ind in range(len(allPosDFs)):
            #gets current position df, scaled current df, and array to help get ppg to normal.
            currDF = allPosDFs[ind]
            scaled = allPosDfsScaled[ind]
            arr = scaleBack[ind]

            #dictionary to store current position grades.
            currPosDict = {}
        
            #iterates through all players in current position.
            for i in range(len(currDF)):

                #gets current information for current player
                currRow = currDF.iloc[[i]]
                currRow = currRow.reset_index()
                scaled = scaled.reset_index()
                scaled = scaled.drop(columns=["index"])
                currRowForModel = scaled.iloc[[i]] 

                pos = currRow.loc[0, "Pos"]
                name = currRow.loc[0, "Name"]
                penalty = currRow.loc[0, "Penalty"]     

                #gets in correct positinal model and predicts each player
                model = modelsDict[pos]
                prediction = model.predict(currRowForModel)  

                #inverse transform the scaled predictions to get the original scale by reversing formula
                prediction = (prediction*(arr[1] - arr[0])) + arr[0]

                #this is where the penalty is added and prediction is entered into the dictionary
                currPosDict[name] = prediction[0]*penalty   

            #appends current position grades to all of them.
            indPosArr.append(currPosDict)

            
    
        #sorts dictionary
        finalqbs = indPosArr[3]

        finalqbs = dict(sorted(finalqbs.items(), key=lambda item: item[1], reverse=True))

        n = len(finalqbs)
        rankNums = np.arange(1, n+1)
        finalqbs = list(finalqbs.keys())
        finalqbs = pd.DataFrame(finalqbs, columns=['Name'])
        finalqbs.insert(0, "Rank", rankNums, True)


        #writes into csv files
        if ppr == 0:
            finalqbs.to_csv("final_rankings/NonPPR_rankings/QBs_NonPPR.csv", encoding='utf-8', index=False)
        elif ppr == 1:
            finalqbs.to_csv("final_rankings/HalfPPR_rankings/QBs_HalfPPR.csv", encoding='utf-8', index=False)
        elif ppr == 2:
            finalqbs.to_csv("final_rankings/PPR_rankings/QBs_PPR.csv", encoding='utf-8', index=False)



scorer()

ValueError: Input X contains infinity or a value too large for dtype('float64').